In [50]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import regex as re

In [135]:
!spacy download en_core_web_sm

In [41]:
main_url = 'https://www.beeradvocate.com/beer/top-rated/'

In [42]:
r = requests.get(main_url)
soup = BeautifulSoup(r.text, 'html.parser')

In [43]:
links = []
table_items = soup.find_all('tr')
for item in table_items:
    if item.find('a'):
        url = 'https://www.beeradvocate.com' + item.find('a')['href']
        links = links + [url]

n = len(links)
print(n)
print(links[:5])

250
['https://www.beeradvocate.com/beer/profile/23222/78820/', 'https://www.beeradvocate.com/beer/profile/26/42349/', 'https://www.beeradvocate.com/beer/profile/17981/110635/', 'https://www.beeradvocate.com/beer/profile/25888/87246/', 'https://www.beeradvocate.com/beer/profile/46317/16814/']


In [128]:
def clean_title(string):
    string = string.replace('\n', ' ').replace('<br>', "")
    string = re.sub(r'[^\w]', ' ', string)
    
    return string

In [131]:
reviews = pd.DataFrame(columns=['product_name','product_review', 'user_rating'])

i = 0
for link in links:
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    product_name = soup.find('div', {'class':'titleBar'}).text
    sub_tile = soup.find('div', {'class':'titleBar'}).find('span').text
    product_name = product_name[:-len(sub_tile)-1]
    product_name = clean_title(product_name)
    
    for review in soup.find_all('div', {'id':'rating_fullview_container'}):
        review_text = clean_title(str(review).split('<br/>')[5])
        user_rating = float(review.find('span', {'class': 'BAscore_norm'}).text)
        reviews.loc[i] = [product_name, review_text, user_rating]
        i += 1

reviews

,product_name,product_review,user_rating
0,Kentucky Brunch Brand Stout,2020 vintage acquired during the pandemic It ...,5.00
1,Kentucky Brunch Brand Stout,Long time waiting to tick this one and I have...,4.56
2,Kentucky Brunch Brand Stout,This review is for the 2019 batch It was bott...,5.00
3,Kentucky Brunch Brand Stout,Supreme maple OD Soooo easy drinking amp we...,5.00
4,Kentucky Brunch Brand Stout,I have now had 4 different years of KBBS and c...,5.00
...,...,...,...
6215,The Streets,Had the good fortune to get 24 of these Tried...,4.85
6216,The Streets,Incredible beer Tasted from can Robust aroma...,5.00
6217,The Streets,Cloudy orange appearance with white head that ...,4.52
6218,The Streets,Can dated 3 20 17 This is the third can consu...,4.75


In [133]:
reviews.to_csv('reviews.csv', index=False)